## Integrate MERFISH experiments after QC filtering

In [ ]:
import scanpy as sc
import scvi
from tqdm.notebook import tqdm
import os
import numpy as np
import pandas as pd
from scvi.model.utils import mde
import matplotlib.pyplot as plt
import pickle
import glob

In [ ]:
sc.set_figure_params(figsize=(4, 4))

Add the paths to all folders for experiments to be integrated

In [ ]:
input_folders = glob.glob("D:/amonell/merscope_final/SI*")

In [ ]:
output_folder = r"D:/amonell/merscope_final/analysis/final"

Run scvi integration

In [ ]:
for path_adata in input_folders:
    rolls = glob.glob(os.path.join(path_adata, "adatas", "05_unrolled*roll_roll*.h5ad"))
    for roll in rolls:
        print(path_adata)
        print(roll)

In [ ]:
adata_list = []
for path_adata in input_folders:
    rolls_ = glob.glob(
        os.path.join(path_adata, "adatas", "05_unrolled*roll_roll*.h5ad")
    )
    for roll in rolls_:
        experiment = sc.read(roll)
        experiment.obs["batch"] = (
            os.path.basename(path_adata) + f"_{experiment.obs.roll.values[0]}"
        )
        adata_list.append(experiment.copy())

concatenated = sc.concat(adata_list)

concatenated.obs_names_make_unique()
concatenated.layers["counts"] = concatenated.X
scvi.model.SCVI.setup_anndata(concatenated, layer="counts", batch_key="batch")
vae = scvi.model.SCVI(concatenated, n_layers=2, n_latent=30, gene_likelihood="nb")
vae.train()

Use integration to calculate new clustering and neighbors

In [ ]:
concatenated.obsm["X_scVI"] = vae.get_latent_representation()

In [ ]:
sc.pp.neighbors(concatenated, use_rep="X_scVI")

Use MDE as new visualization. Can sometimes give extreme outlier values so I put some filters on at 8 units in all directions

In [ ]:
concatenated.obsm["X_mde"] = mde(concatenated.obsm["X_scVI"])

In [ ]:
concatenated.obs["new_batch"] = concatenated.obs.batch.values

In [ ]:
xcoords = concatenated.obsm["X_mde"][:, 0]
ycoords = concatenated.obsm["X_mde"][:, 1]
category_names = concatenated.obs.new_batch.values

In [ ]:
sc.tl.leiden(concatenated)

In [ ]:
os.mkdir(output_folder)

In [ ]:
concatenated.write(os.path.join(output_folder, "concatenated_integrated.h5ad"))

In [ ]:
sc.set_figure_params(figsize=(5, 5))
sc.pl.embedding(concatenated, basis="mde", color=["leiden"])

## Save our SCVI VAE

In [ ]:
import pickle

# File path where you want to save the object
file_path = os.path.join(output_folder, "vae_object.pkl")

# Dump the object to the file
with open(file_path, "wb") as file:
    pickle.dump(vae, file)

## Subcluster data

In [ ]:
def get_celltype(celltype, ad_sp):
    ctype = ad_sp[ad_sp.obs.leiden.isin([celltype])]
    sc.pp.neighbors(ctype, use_rep="X_scVI")
    sc.tl.leiden(ctype, resolution=0.5)
    sc.tl.umap(ctype)
    return ctype


def reunite_with_ad(ad_sp, subset_ad, celltype):
    new_labels = []
    subclusters = ad_sp.obs.Sub_leiden.values
    clusters = ad_sp.obs.leiden.values
    idex = ad_sp.obs.index.values
    for i in tqdm(range(len(subclusters))):
        if clusters[i] == celltype:
            new_labels.append(subset_ad.obs.loc[idex[i], :].leiden)
        else:
            new_labels.append(subclusters[i])
    ad_sp.obs.Sub_leiden = new_labels
    return ad_sp

In [ ]:
ad_sp = sc.read(os.path.join(output_folder, "concatenated_integrated.h5ad"))
ad_sp.obs["Sub_leiden"] = ad_sp.obs["leiden"]

In [ ]:
lts = ad_sp.obs.leiden.cat.categories
for leiden_to_subset in lts:
    mac = get_celltype(leiden_to_subset, ad_sp)
    sc.pl.embedding(mac, basis="mde", color=["leiden"], vmax=1, size=0.1)
    mac.obs["leiden"] = [leiden_to_subset + "_" + i for i in mac.obs.leiden]
    ad_sp = reunite_with_ad(ad_sp, mac, leiden_to_subset)
    sc.pl.embedding(ad_sp, basis="mde", color=["Sub_leiden"])

In [ ]:
ad_sp.write(os.path.join(output_folder, "integrated_clustered.h5ad"))